# Fault Demo

In this demo notebook, we will go over how to use the fault model in Open Seismic with the F3 data block (real seismic data). Note that we will be drawing on knowledge that is introduced in the Open Seismic examples. If you have not reviewed those examples, please review them first and come back. 

## Sections
Fault.1 **Using the Default Fault Model**<br/>
Fault.2 **Using the Fault Model with Custom Model Script**<br/>
Fault.3 **Using the Fault Model with Custom Processor Scripts**

## Imports

Here are some initial imports and global variables that we need to set up before moving forward. Make sure to run `pip install -r requirements.txt` and build/pull the Docker image for Open Seismic.

In [1]:
import json
from pathlib import PurePath
import os
import numpy as np
import matplotlib.pyplot as plt

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        if 'ipynb' in root or 'ipynb' in dirs or 'ipynb' in files:
            continue
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
            
open_seismic_docker = 'open_seismic'
assets_path = PurePath(os.getcwd()).parent.joinpath('fault')
os_path = PurePath(os.getcwd()).parent.parent

models_path = os_path.joinpath('models')
#models_mount = f' -v {models_path}:/core/python/models/'#
models_mount = f' -v {models_path}:/core/models/'

## Section Fault.1: Using the Default Fault Model

In this section, we will go over how to use the default fault model that exists within Open Seismic. 

### Steps
1. Construct the JSON config file for directing Open Seismic to the necessary files. For this demo, we have already constructed the JSON configuration. Run the cell below to display the JSON config file. We will be using coarse cube inference as our inference task of choice because this aligns with our intention of inference over a large piece of data.

In [2]:
json_path = assets_path.joinpath('fault_config.json')
with open(str(json_path), 'rb') as f:
    json_config = json.load(f)
print(json.dumps(json_config, indent=4))

{
    "inference_params": {
        "data": "data_mnt/Dutch_F3_data/",
        "given_model": "fseg",
        "infer_type": "coarse_cube_sync",
        "output": "fault_demo_output"
    },
    "visualize_params": {
        "input": "fault_demo_output",
        "output": "visualization",
        "model_type": "fault",
        "slice_no": "100"
    }
}


os_pathAfter running the cell, you might notice that we direct Open Seismic to look at the `data_mnt` directory instead of a common directory that we have recommended. The reason for this is because in this demo, we will be using the F3 data block, which a particularly large file and should not be copied multiple times. Additionally, Docker does not support symlinks on certain operating systems. Therefore, we will use one more mount command to mount the F3 data block into the right directory in Open Seismic's Docker container.

2. Construct the command to run Open Seismic.

In [3]:
!pwd

/home/sdp/ravi/open_seismic/models/demos


In [4]:
local_dir = assets_path
run_dir = assets_path.joinpath('runs')
data_dir = os_path.joinpath('data', '')
os_input = '/core/mnt/fault_config.json'

infer_mount = f'-v {local_dir}:/core/mnt/'
runs_mount = f'-v {run_dir}:/core/runs/'
data_mount = f'-v {data_dir}:/core/data_mnt/'
mounts = f'{infer_mount} {data_mount} {runs_mount} {models_mount}'
os_cmd = f"docker run {mounts} {open_seismic_docker} ./run.sh -c {os_input}"
os_cmd

'docker run -v /home/sdp/ravi/open_seismic/models/fault:/core/mnt/ -v /home/sdp/ravi/open_seismic/data:/core/data_mnt/ -v /home/sdp/ravi/open_seismic/models/fault/runs:/core/runs/  -v /home/sdp/ravi/open_seismic/models:/core/models/ open_seismic ./run.sh -c /core/mnt/fault_config.json'

3. Run Open Seismic!

In [5]:
! {os_cmd}

[ARG PARSE] model= data=data_mnt/Dutch_F3_data/ output=runs/February25_07-27-06_AM_2021/fault_demo_output
[OUTPUT] Making new folder for output storage.
[INFER] Setting up inference...
[SWAP] Preprocessor not found. Defaulting to given model preprocess.
[SWAP] Postprocessor not found. Defaulting to given model postprocess.
[SWAP] Custom bin, xml, and model init not found. Using default model init with given model.
[INFER] Using model: fseg_model
[LOADER] Loading file: data_mnt/Dutch_F3_data//f3_8bit.segy
[INFER] Conducting inference...
[INFER] Reshaping inference input...
[INFER] Conducting inference on input: f3_8bit.segy...
1320it [02:20,  9.40it/s][INFER] Saving output to output path: runs/February25_07-27-06_AM_2021/fault_demo_output/f3_8bit.segy-input/out.npy
[INFER] Complete!

[VISUALIZATION] Data file runs/February25_07-27-06_AM_2021/fault_demo_output/f3_8bit.segy-input/out.npy -  loaded
[VISUALIZATION] Visualization complete! File saved to: runs/February25_07-27-06_AM_2021/visu

## Section Fault.2: Using the Fault with Custom Model Script

In this section, we will swap out the default model script with our own custom one. You can find the model script in `demos/assets/fault_demo/only_model_script`. 

### Steps
1. Construct the JSON config file for directing Open Seismic to the necessary files. For this demo, we have already constructed the JSON configuration. Run the cell below to display the JSON config file. We will be using coarse cube inference as our inference task of choice because this aligns with our intention of inference over a large piece of data.

In [6]:
json_path = assets_path.joinpath('fault_model_config.json')
with open(str(json_path), 'rb') as f:
    json_config = json.load(f)
print(json.dumps(json_config, indent=4))

{
    "inference_params": {
        "data": "data_mnt/Synthetic_fault_data/seis/",
        "model": "models/fault/only_model_script/",
        "given_model": "fseg",
        "infer_type": "sync",
        "output": "fault_model_demo_output"
    }
}


Notice that we have specified the model script in the `model` parameter of the JSON config file, but we have kept the `fseg` option in the `given_model` parameter. In terms of priority, Open Seismic will look at the path specified in `model` first, then will look at the `given_model` files to fill in the required files.

2. Construct the command to run Open Seismic.

In [7]:
local_dir = assets_path
run_dir = assets_path.joinpath('runs')
data_dir = os_path.joinpath('data', '')
os_input = '/core/mnt/fault_model_config.json'

infer_mount = f'-v {local_dir}:/core/mnt/'
runs_mount = f'-v {run_dir}:/core/runs/'
data_mount = f'-v {data_dir}:/core/data_mnt/'
mounts = f'{infer_mount} {data_mount} {runs_mount} {models_mount}'
os_cmd = f"docker run {mounts} {open_seismic_docker} ./run.sh -c {os_input}"
os_cmd

'docker run -v /home/sdp/ravi/open_seismic/models/fault:/core/mnt/ -v /home/sdp/ravi/open_seismic/data:/core/data_mnt/ -v /home/sdp/ravi/open_seismic/models/fault/runs:/core/runs/  -v /home/sdp/ravi/open_seismic/models:/core/models/ open_seismic ./run.sh -c /core/mnt/fault_model_config.json'

3. Run Open Seismic!

In [8]:
! {os_cmd}

[ARG PARSE] model=models/fault/only_model_script/ data=data_mnt/Synthetic_fault_data/seis/ output=runs/February25_07-29-38_AM_2021/fault_model_demo_output
[OUTPUT] Making new folder for output storage.
[SWAP] Preprocessor not found. Defaulting to given model preprocess.
[SWAP] Postprocessor not found. Defaulting to given model postprocess.
[SWAP] Found custom model script.
[SWAP] Custom bin or xml not found. Using given model xml and bin with custom model init.
[INFER] Using model: fseg_model
[INFER] Loading datafile names for inference...
[LOADER] Loading file: data_mnt/Synthetic_fault_data/seis//1.dat
[LOADER] Loading file: data_mnt/Synthetic_fault_data/seis//2.dat
[LOADER] Loading file: data_mnt/Synthetic_fault_data/seis//3.dat
[LOADER] Loading file: data_mnt/Synthetic_fault_data/seis//0.dat
[LOADER] Loading file: data_mnt/Synthetic_fault_data/seis//4.dat
[INFER] Conducting inference...
100%|##########| 5/5 [00:00<00:00,  6.76it/s][INFER] Complete!



## Section Fault.3: Using the Fault with Custom Processor Scripts

In this section, we will swap out the default processor scripts with our own custom ones. You can find the model script in `demos/assets/fault_demo/only_processor_scripts`. 

### Steps
1. Construct the JSON config file for directing Open Seismic to the necessary files. For this demo, we have already constructed the JSON configuration. Run the cell below to display the JSON config file. We will be using coarse cube inference as our inference task of choice because this aligns with our intention of inference over a large piece of data.

In [9]:
json_path = assets_path.joinpath('fault_proc_config.json')
with open(str(json_path), 'rb') as f:
    json_config = json.load(f)
print(json.dumps(json_config, indent=4))

{
    "inference_params": {
        "data": "data_mnt/Synthetic_fault_data/seis/",
        "model": "models/fault/only_processor_scripts/",
        "given_model": "fseg",
        "infer_type": "sync",
        "output": "fault_proc_demo_output"
    }
}


Notice that we have specified the model script in the `model` parameter of the JSON config file, but we have kept the `fseg` option in the `given_model` parameter. In terms of priority, Open Seismic will look at the path specified in `model` first, then will look at the `given_model` files to fill in the required files.

2. Construct the command to run Open Seismic.

In [10]:
local_dir = assets_path
run_dir = assets_path.joinpath('runs')
data_dir = os_path.joinpath('data', '')
os_input = '/core/mnt/fault_proc_config.json'

infer_mount = f'-v {local_dir}:/core/mnt/'
runs_mount = f'-v {run_dir}:/core/runs/'
data_mount = f'-v {data_dir}:/core/data_mnt/'
mounts = f'{infer_mount} {data_mount} {runs_mount} {models_mount}'
os_cmd = f"docker run {mounts} {open_seismic_docker} ./run.sh -c {os_input}"
os_cmd

'docker run -v /home/sdp/ravi/open_seismic/models/fault:/core/mnt/ -v /home/sdp/ravi/open_seismic/data:/core/data_mnt/ -v /home/sdp/ravi/open_seismic/models/fault/runs:/core/runs/  -v /home/sdp/ravi/open_seismic/models:/core/models/ open_seismic ./run.sh -c /core/mnt/fault_proc_config.json'

3. Run Open Seismic!

In [11]:
! {os_cmd}

[ARG PARSE] model=models/fault/only_processor_scripts/ data=data_mnt/Synthetic_fault_data/seis/ output=runs/February25_07-29-42_AM_2021/fault_proc_demo_output
[OUTPUT] Making new folder for output storage.
[SWAP] Found custom preporcessing script.
[SWAP] Found custom postporcessing script.
[SWAP] Custom bin, xml, and model init not found. Using default model init with given model.
[INFER] Using model: fseg_model
[INFER] Loading datafile names for inference...
[LOADER] Loading file: data_mnt/Synthetic_fault_data/seis//1.dat
[LOADER] Loading file: data_mnt/Synthetic_fault_data/seis//2.dat
[LOADER] Loading file: data_mnt/Synthetic_fault_data/seis//3.dat
[LOADER] Loading file: data_mnt/Synthetic_fault_data/seis//0.dat
[LOADER] Loading file: data_mnt/Synthetic_fault_data/seis//4.dat
[INFER] Conducting inference...
100%|##########| 5/5 [00:00<00:00,  6.91it/s][INFER] Complete!



Congratulations! You now know how to use the fault model in Open Seismic.

## Summary
In this demo, you learned about:
1. Using the default fault model with your data
2. Using the fault model with your data and model script
3. Using the fault model with your data and processor scripts

We haven't talked about combining custom model and processor scripts together, but this can be done as well by storing the model and processor scripts in one mount directory. If you would like to learn more about the other models, please go to their respective folders and run their demos.